In [331]:
import pandas as pd
import numpy as np
import nltk
import re

In [332]:
X_train=pd.read_csv('/home/hduser/Desktop/FundingProject/Data/train.csv',header=0)
X_test=pd.read_csv('/home/hduser/Desktop/FundingProject/Data/test.csv',header=0)

In [338]:
X_train.drop(['name','disable_communication','country','currency','backers_count','final_status'],axis=1,inplace=True)

In [339]:
X_test.drop(['name','disable_communication','country','currency'],axis=1,inplace=True)

In [348]:
def combinedata():
    combined_X=X_train.append(X_test)
    #combined_X['desc']=combinedX['desc'].astype(str)
    combined_X['diff_creat_laun']=abs(combined_X['created_at']-combined_X['launched_at'])
    combined_X['diff_laun_status']=abs(combined_X['launched_at']-combined_X['state_changed_at'])
    combined_X['diff_status_dead']=(combined_X['state_changed_at']-combined_X['deadline'])
    print("Shape of total data",combined_X.shape)
    combined_X.reset_index(inplace=True)
    combined_X.drop('index',axis=1,inplace=True)
    return combined_X

In [349]:
combinedX=pd.DataFrame(combinedata())

Shape of total data (171594, 11)


In [351]:
combinedX.drop(['created_at','launched_at','state_changed_at','deadline'],axis=1,inplace=True)

In [354]:
combinedX.isnull().any() #Null values check

project_id          False
desc                 True
goal                False
keywords            False
diff_creat_laun     False
diff_laun_status    False
diff_status_dead    False
dtype: bool

## Cleaning

In [355]:
kickdesc = pd.Series(X_train['desc'].tolist() + X_test['desc'].tolist()).astype(str)

In [356]:
# this function cleans punctuations, digits and irregular tabs. Then converts the sentences to lower
def desc_clean(word):
    p1 = re.sub(pattern='(\W+)|(\d+)|(\s+)',repl=' ',string=word)
    p1 = p1.lower()
    return p1


In [357]:
kickdesc=kickdesc.map(desc_clean)

In [358]:
from nltk.corpus import stopwords
stop =(stopwords.words('english'))
stop.extend(['mr','my','the','a','may','also','zero','one','two','three','four','five','six','seven','eight','nine','ten','across','among','beside','however','yet','within'])

In [359]:
kickdesc = [[x for x in x.split() if x not in stop] for x in kickdesc]

In [360]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='english')
kickdesc = [[stemmer.stem(x) for x in x] for x in kickdesc]

In [361]:
stemmer = SnowballStemmer(language='english')
kickdesc = [[stemmer.stem(x) for x in x] for x in kickdesc]

In [362]:
kickdesc = [[x for x in x if len(x) > 2] for x in kickdesc]

In [363]:
kickdesc = [' '.join(x) for x in kickdesc]

In [364]:
kickkey=combinedX.keywords.astype(str)

In [365]:
kickkey=kickkey.map(desc_clean)

In [366]:
kickkey = [[x for x in x.split() if x not in stop] for x in kickkey]

In [367]:
stemmer = SnowballStemmer(language='english')
kickkey = [[stemmer.stem(x) for x in x] for x in kickkey]

In [368]:
stemmer = SnowballStemmer(language='english')
kickkey = [[stemmer.stem(x) for x in x] for x in kickkey]

In [369]:
kickkey = [[x for x in x if len(x) > 2] for x in kickkey]

In [370]:
kickkey = [' '.join(x) for x in kickkey]

In [371]:
type(kickkey)

list

In [372]:
dframe1=pd.DataFrame({'Description':kickdesc}).astype(str)

In [373]:
dframe2=pd.DataFrame({'Key':kickkey}).astype(str)

In [374]:
#result = pd.concat([dframe1, dframe2], axis=1, join_axes=[dframe1.index])

In [375]:
dframe=[]
for i in range(dframe1.shape[0]):
    dframe.append(dframe1.Description[i]+" "+dframe2.Key[i])

In [376]:
#dframe=pd.DataFrame({'Desc_Key':dframe}).astype(str)

In [377]:
#dframe.head(2)

In [378]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
cv = CountVectorizer(max_features=650)
#cv = TfidfVectorizer(max_features=650) #67

In [379]:
alldesc = cv.fit_transform(dframe).todense()

In [380]:
#create a data frame
combine = pd.DataFrame(alldesc)
combine.rename(columns= lambda x: 'variable_'+ str(x), inplace=True)

In [381]:
type(combine)

pandas.core.frame.DataFrame

In [382]:
combinedX.tail(1)

,project_id,desc,goal,keywords,diff_creat_laun,diff_laun_status,diff_status_dead
171593,kkst1164902454,"Join Daisy, on her adventures, in this theme b...",850.0,daisys-sensory-adventures-childrens-book,5455,1296000,0


In [383]:
combinedX.drop(['project_id','desc','keywords'], axis=1,inplace=True)

In [386]:
from sklearn.preprocessing import MinMaxScaler
combinedX['goal']=MinMaxScaler().fit_transform(combinedX['goal'])
combinedX['diff_creat_laun']=MinMaxScaler().fit_transform(combinedX['diff_creat_laun'])
combinedX['diff_laun_status']=MinMaxScaler().fit_transform(combinedX['diff_laun_status'])
combinedX['diff_status_dead']=MinMaxScaler().fit_transform(combinedX['diff_status_dead'])

/home/hduser/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/hduser/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/hduser/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
/home/hduser/anacond

### Now ready to train model!!!

In [391]:
cols_to_use = ['goal','diff_creat_laun','diff_laun_status','diff_status_dead']

In [392]:
train1=pd.read_csv('/home/hduser/Desktop/FundingProject/Data/train.csv')

In [393]:
#train1.project_id[0]

In [394]:
target = train1['final_status']

In [395]:
target.head()

0    1
1    0
2    0
3    1
4    0
Name: final_status, dtype: int64

In [396]:
#Numerical features normalized
train = combinedX.ix[0:108128]
test = combinedX.ix[108129:]
#test.reset_index(drop=True,inplace=True)

/home/hduser/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


In [397]:
print(train.shape)
print('--------')
print(test.shape)

(108129, 4)
--------
(63465, 4)


In [398]:
#train1.shape[1]

In [399]:
#split the text features

train_text = combine[:108129]
test_text = combine[108129:]

test_text.reset_index(drop=True,inplace=True)

In [400]:
print(train_text.shape)
print(test_text.shape)

(108129, 650)
(63465, 650)


In [402]:
X_train1 = train.join(train_text)
X_test1 = test.join(test_text)
#X_train1=train_text.join(train)
#X_test1 =test_text.join(test)
#X_train1

In [306]:
import xgboost as xgb

In [307]:
dtrain = xgb.DMatrix(data=X_train1, label = target)
dtest = xgb.DMatrix(data=X_test1)

In [308]:
params = {
    'objective':'binary:logistic',
    'eval_metric':'error',
    'eta':0.025,
    'max_depth':6,
    'subsample':0.7,
    'colsample_bytree':0.7,
    'min_child_weight':5
    
}

In [309]:
bst = xgb.cv(params, dtrain, num_boost_round=1000, early_stopping_rounds=40,nfold=5,verbose_eval=10)

[0]	train-error:0.309824+0.00101828	test-error:0.313582+0.0022737
[10]	train-error:0.306169+0.00235387	test-error:0.309114+0.00203822
[20]	train-error:0.303866+0.0021907	test-error:0.30757+0.00242637
[30]	train-error:0.302169+0.00095206	test-error:0.306516+0.00300895
[40]	train-error:0.300779+0.000567382	test-error:0.305582+0.00305864
[50]	train-error:0.299249+0.000421263	test-error:0.304287+0.00279358
[60]	train-error:0.297184+0.000284684	test-error:0.302566+0.00308037
[70]	train-error:0.295917+0.000283908	test-error:0.301558+0.00256077
[80]	train-error:0.294065+0.000443302	test-error:0.30018+0.00218066
[90]	train-error:0.292661+0.000357765	test-error:0.299052+0.00232434
[100]	train-error:0.291586+0.00045087	test-error:0.298247+0.00228554
[110]	train-error:0.290442+0.000424417	test-error:0.29711+0.00203453
[120]	train-error:0.289323+0.000518916	test-error:0.296278+0.0021184
[130]	train-error:0.288337+0.000699515	test-error:0.295371+0.00257814
[140]	train-error:0.287066+0.000855294	tes

In [310]:
bst_train = xgb.train(params, dtrain, num_boost_round=1000)

In [318]:
p_test = bst_train.predict(dtest)

In [324]:
len(p_test)

63465

In [325]:
sub = pd.DataFrame()
sub['project_id'] = X_test['project_id']


In [328]:
sub['final_status']=p_test

In [329]:
sub['final_status'] = [1 if x > 0.5 else 0 for x in sub['final_status']]

In [330]:
sub.to_csv("/home/hduser/Desktop/FundingProject/Result/xgb_with_python_feats.csv",index=False) 